# **Memory & Chain**

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

sk-proj-9CjlFKtZ98k4zSr3ftdDT3BlbkFJ4GKgyfKPxApbBkfwoLMn


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Memory**

### (1) 대화 기록 저장하기

In [3]:
# 메모리 선언하기(초기화)
memory = ConversationBufferMemory(return_messages=True)

# 저장
memory.save_context({"input": "안녕하세요!"},
                    {"output": "안녕하세요! 어떻게 도와드릴까요?"})

memory.save_context({"input": "메일을 써야하는데 도와줘"},
                    {"output": "누구에게 보내는 어떤 메일인가요?"})

# 현재 담겨 있는 메모리 내용 전체 확인
memory.load_memory_variables({})

{'history': [HumanMessage(content='안녕하세요!'),
  AIMessage(content='안녕하세요! 어떻게 도와드릴까요?'),
  HumanMessage(content='메일을 써야하는데 도와줘'),
  AIMessage(content='누구에게 보내는 어떤 메일인가요?')]}

### (2) GPT와 대화 기록 저장하기

#### 1) 대화 준비

In [4]:
k = 3

# Chroma 데이터베이스 인스턴스
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)

# retriever 선언
retriever = database.as_retriever(search_kwargs={"k": k})

# ChatOpenAI 선언
chat = ChatOpenAI(model="gpt-3.5-turbo")

# RetrievalQA 선언
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True)

#### 2) 대화 시도 및 기록

In [5]:
# 질문 답변1
query = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query)
result['result']

'생성형 AI 프로젝트를 도입할 때 예상되는 보안 위협은 악의적인 공격에 노출될 수 있는 위협이 크게 포함됩니다. 이에는 프로젝트의 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하는 경우, 훈련 데이터나 모델 자체를 오염시키는 경우, 모델을 재학습시켜 사용자 인프라에 침입하는 경우, 가짜 뉴스나 잘못된 정보로 모델을 훈련시키는 경우 등이 있을 수 있습니다.'

In [6]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트를 도입할 때 예상되는 보안 위협은 악의적인 공격에 노출될 수 있는 위협이 크게 포함됩니다. 이에는 프로젝트의 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하는 경우, 훈련 데이터나 모델 자체를 오염시키는 경우, 모델을 재학습시켜 사용자 인프라에 침입하는 경우, 가짜 뉴스나 잘못된 정보로 모델을 훈련시키는 경우 등이 있을 수 있습니다.')]}

#### 3) 이어지는 질문과 답변

In [7]:
# 질문 답변2
query = "훈련 데이터나 가중치를 오염시키는게 무슨 의미야?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트를 도입할 때 예상되는 보안 위협은 악의적인 공격에 노출될 수 있는 위협이 크게 포함됩니다. 이에는 프로젝트의 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하는 경우, 훈련 데이터나 모델 자체를 오염시키는 경우, 모델을 재학습시켜 사용자 인프라에 침입하는 경우, 가짜 뉴스나 잘못된 정보로 모델을 훈련시키는 경우 등이 있을 수 있습니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 악의적인 공격자가 AI 모델을 속이거나 손상시키기 위해 훈련 데이터나 모델의 가중치를 변경하여 모델의 동작을 변형시키는 것을 의미합니다. 이로 인해 모델이 잘못된 판단을 내리거나 의도하지 않은 결과를 출력할 수 있게 되며, 이는 모델의 성능을 손상시키고 사용자에게 해를 끼칠 수 있습니다.')]}

In [8]:
# 질문 답변3
query = "이를 방지하기 위해 어떻게 해야 해?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트를 도입할 때 예상되는 보안 위협은 악의적인 공격에 노출될 수 있는 위협이 크게 포함됩니다. 이에는 프로젝트의 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하는 경우, 훈련 데이터나 모델 자체를 오염시키는 경우, 모델을 재학습시켜 사용자 인프라에 침입하는 경우, 가짜 뉴스나 잘못된 정보로 모델을 훈련시키는 경우 등이 있을 수 있습니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 악의적인 공격자가 AI 모델을 속이거나 손상시키기 위해 훈련 데이터나 모델의 가중치를 변경하여 모델의 동작을 변형시키는 것을 의미합니다. 이로 인해 모델이 잘못된 판단을 내리거나 의도하지 않은 결과를 출력할 수 있게 되며, 이는 모델의 성능을 손상시키고 사용자에게 해를 끼칠 수 있습니다.'),
  HumanMessage(content='이를 방지하기 위해 어떻게 해야 해?'),
  AIMessage(content='위의 문맥에서는 오픈소스 AI 모델을 이용한 악의적인 공격에 대해 다루고 있습니다. 이를 방지하기 위해서는 다음과 같은 조치를 취할 수 있습니다:\n1. 오픈소스 AI 모델을 사용할 때는 신뢰할 수 있는 소스에서 다운로드하고 사용해야 합니다.\n2. 모델을 사용할 때 보안 전문가의 조언을 받아야 합니다.\n3. 모델을 테스트하고 모니터링하여 의심스러운 활동을 식별해야 합니다.\n4. 모델이나 지원 코드의 보안 취약점을 식별하고 보완해야 합니다.\n5. 엔터프라이즈 버전의 제품을 구매하거나, 내부에 보호 장치를 마련하여 외부 공격에 대비해야 합니다. \n\n그러나 전문적인 안전 조치를 위해 보안 전문가와 협력하는 것이 가장 효과적일 수 있습니다.')]}

* 훈련 데이터 오염을 방지하기 위한 대책을 물었으나, 일반적인 보안 위협 방지 대책을 이야기 함.
* 맥락을 유지하기 위해서 메시지의 내용을 프롬프트에 포함시켜야 함.
    * 이를 손쉽게 엮어주는 방법 **Chain**

## **3.Chain**

* 절차 다시 정리
    * 질문을 받아
    * 유사도 높은 문서를 DB에서 검색(RAG)
    * 이전 대화 내용을 메모리에서 읽어오기
    * [질문 + 유사도높은 문서 + 이전 대화내용]으로 프롬프트 구성
    * GPT에 질문하고 답변 받기
* Chain
    * 이러한 절차를 코드로 하나하나 엮고, 프롬프트를 구성하는 코드는 상당히 복잡합니다.
    * 랭체인에서 제공하는 Chain 함수를 이용하면 쉽게 구현 가능!  

### (1) Chain 함수로 연결하기
* **ConversationalRetrievalChain**

In [9]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")

### (2) 사용하기

#### 1) 첫번째 질문

In [10]:
# 첫번째 질문
query1 = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query1)
result['answer']

'생성형 AI 프로젝트는 악의적인 공격에 노출될 수 있는 다양한 위협이 있습니다. 예를 들어, 해커가 프로젝트 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치 등을 오염시킬 수 있습니다. 또한, 모델을 재훈련시켜 사용자 인프라에 침입하거나 가짜 뉴스, 잘못된 정보로 모델을 훈련시키는 위협도 있을 수 있습니다. 따라서 보안 측면에서 주의가 필요합니다.'

In [11]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트는 악의적인 공격에 노출될 수 있는 다양한 위협이 있습니다. 예를 들어, 해커가 프로젝트 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치 등을 오염시킬 수 있습니다. 또한, 모델을 재훈련시켜 사용자 인프라에 침입하거나 가짜 뉴스, 잘못된 정보로 모델을 훈련시키는 위협도 있을 수 있습니다. 따라서 보안 측면에서 주의가 필요합니다.')]}

#### 2) 두번째 질문

In [12]:
# 두번째 질문
query2 = "모델을 재학습시면 어떤 문제가 발생되는거야?"
result = qa(query2)
result['answer']

'재학습된 모델에 대한 보안 위협은 다양한 형태로 나타날 수 있습니다. 예를 들어, 해커가 악성 코드를 모델에 주입하여 사용자의 인프라에 침입할 수 있습니다. 또한 가짜 뉴스나 잘못된 정보로 모델을 훈련시켜 분류 오류를 유발하거나 사용자에게 잘못된 정보를 제공할 수도 있습니다. 이는 모델이 잘못된 결정을 내리거나 부적절한 행동을 유도할 수 있어 보안 문제를 야기할 수 있습니다.'

In [13]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트는 악의적인 공격에 노출될 수 있는 다양한 위협이 있습니다. 예를 들어, 해커가 프로젝트 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치 등을 오염시킬 수 있습니다. 또한, 모델을 재훈련시켜 사용자 인프라에 침입하거나 가짜 뉴스, 잘못된 정보로 모델을 훈련시키는 위협도 있을 수 있습니다. 따라서 보안 측면에서 주의가 필요합니다.'),
  HumanMessage(content='모델을 재학습시면 어떤 문제가 발생되는거야?'),
  AIMessage(content='재학습된 모델에 대한 보안 위협은 다양한 형태로 나타날 수 있습니다. 예를 들어, 해커가 악성 코드를 모델에 주입하여 사용자의 인프라에 침입할 수 있습니다. 또한 가짜 뉴스나 잘못된 정보로 모델을 훈련시켜 분류 오류를 유발하거나 사용자에게 잘못된 정보를 제공할 수도 있습니다. 이는 모델이 잘못된 결정을 내리거나 부적절한 행동을 유도할 수 있어 보안 문제를 야기할 수 있습니다.')]}

### (3) 반복문 안에서 질문답변 이어가기

In [14]:
while True:
    query = input('질문 > ')
    query = query.strip()
    print(f'질문 : {query}')
    print('-' * 20)
    if len(query) == 0:
        break
    result = qa({"question": query})
    print(f'답변 : {result["answer"]}')
    print('=' * 50)

질문 : 어떤 보안 위협이 있어?
--------------------
답변 : AI 모델을 보안 측면에서 고려할 때 몇 가지 주의해야 할 점이 있습니다. 첫째로, 오픈소스 AI 모델은 전문성 부족으로 인해 보안 취약점이 발생할 수 있습니다. 따라서 보안 전문가의 조언을 듣고 모델을 테스트하고 평가하는 것이 중요합니다. 둘째로, 악의적인 공격자가 모델을 해킹하거나 조작할 수 있으므로, 모델을 실행하는 환경과 시스템을 보호하는 것이 중요합니다. 또한, 모델의 출력물에 워터마크를 추가하더라도 공격을 막지 못할 수 있으니, 추가적인 보안 조치가 필요할 수 있습니다. 마지막으로, 공격자가 모델을 이용하여 시스템을 과부하시키는 등의 공격을 시도할 수 있으므로, 이러한 공격에 대비하는 방법을 마련해야 합니다.
질문 : 악의적인 공격이 무슨 의미야?
--------------------
답변 : 악의적인 공격은 악의를 갖고 다른 사람, 기업, 시스템 또는 네트워크에 해를 가하기 위해 의도적으로 실행되는 공격을 말합니다. 이러한 공격은 주로 해킹, 데이터 유출, 시스템 마비, 개인 정보 도용 등을 목적으로 하는 악의적인 행위를 포함합니다.
질문 : 선의를 품은 공격도 있어?
--------------------
답변 : 예, 선의를 품은 공격도 있습니다. 선의를 품은 공격은 해커가 시스템의 보안 취약점을 찾아내어 해당 기업이나 조직에 그 취약점을 알리는 것을 의미합니다. 이는 보안 취약점을 해결하고 더 나은 보안 대책을 마련할 수 있도록 도와주는 행위입니다.
질문 : 
--------------------
